In [1]:
import pandas as pd
import os
import json

In [13]:
bip_df = pd.read_csv(r"C:\\Users\\VinodKumarM\\OneDrive - EQUINIX\\Projects\\fusion_automate_project\\resources\\bip_list.csv")

In [16]:
bip_df.shape[0]

1

In [17]:
df = pd.read_csv('gs://helix-finance-test/helix_automation_resource/BIP_REQUIRED_TABLE_COLUMNS.csv')

In [18]:
ora_cols = pd.read_csv('gs://helix-finance-test/helix_automation_resource/BIP_FUSION_TABLE_SCHEMA.csv')

In [19]:
table_names = list(df.TABLE.unique())

In [20]:
table_names

['HZ_CUST_SITE_USES_ALL']

In [21]:
for table in table_names:
    filtered_df = df.loc[df.TABLE==table]
    print(filtered_df)
    print(ora_cols)
    merge_df = pd.merge(filtered_df,ora_cols,how='left',right_on=['TABLE_NAME','COLUMN_NAME'], left_on=['TABLE','COLUMN'])
    merge_df = merge_df.loc[:,['TABLE','COLUMN', 'DATA_TYPE','DATA_LENGTH']]
    #print(merge_df)
    datatype_map = {'BINARY': 'STRING', 'CLOB': 'STRING', 'NUMERIC': 'NUMERIC', 'VARCHAR': 'STRING', 'DATE': 'TIMESTAMP',
                    'TIMESTAMP': 'TIMESTAMP', 'FLOAT': 'NUMERIC','NUMBER':'NUMERIC','VARCHAR2': 'STRING','TIMESTAMP(6)':'TIMESTAMP'}
    
    _metadata_insert_time = "_metadata_insert_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n"
    
    # To add platform supported column in RAW tables
    raw_meta_cols = "_metadata_insert_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, \n" \
                    "_metadata_update_time TIMESTAMP, \n" \
                    "_metadata_deleted STRING DEFAULT 'N',\n" \
                    "_metadata_dag_run_id STRING,\n"
    
    hist_meta_cols = "_metadata_insert_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, \n" \
                     "_metadata_deleted STRING DEFAULT 'N',\n" \
                     "_metadata_dag_run_id STRING,\n"

    merge_df['bq_type'] = merge_df['DATA_TYPE'].map(datatype_map)
    merge_df.loc[merge_df.COLUMN=='RUN_DATE',["bq_type"]] = 'TIMESTAMP'
    merge_df.loc[merge_df.COLUMN=='KEY',["bq_type"]] = 'STRING'
    merge_df.loc[merge_df.COLUMN=='COL_FOR_SINGLE_FILE',["bq_type"]] = 'STRING'
    #print(merge_df)
    merge_df['ddl'] = merge_df['COLUMN'] + ' ' + merge_df['bq_type'] + ',\n'
    collist = list(merge_df['ddl'].drop_duplicates())
    colstr = ''.join(collist)

    # Stg table ddl creation
    stg_table_name = 'STG_MS_'+table+'_BIP'
    create = 'CREATE TABLE IF NOT EXISTS `helix-data-dev.' + 'helix_fin_ms_stg' + '.' + stg_table_name + '`\n(\n'
    colstrstg = create + colstr + _metadata_insert_time
    colstrstg = colstrstg[:-2] + '\n);'

    
    # Raw table ddl creation
    raw_table_name = 'MS_'+table+'_BIP'
    createraw = 'CREATE TABLE IF NOT EXISTS `helix-data-dev.' + 'helix_fin_ms_raw' + '.' + raw_table_name + '`\n(\n'
    createraw = createraw + colstr + raw_meta_cols
    colstrraw = createraw[:-2] + '\n)\nPARTITION BY DATE(_metadata_update_time);'

    # Hist table ddl creation
    #createhist = 'CREATE TABLE IF NOT EXISTS `project_name.' + pc.HIST_SCHEMA + '.' + hist_table_name + '`\n(\n'
    #createhist = createhist + colstrhist + hist_meta_cols
    #colstrhist = createhist[:-2] + '\n)\nPARTITION BY DATE(_metadata_insert_time);'

    
    with open(os.path.join(r'C:\Users\VinodKumarM\OneDrive - EQUINIX\Projects\fusion_automate_project\fusbq\helix_automation_output', 'STG', stg_table_name + '.sql'), 'w') as file:
        file.write(colstrstg)

    with open(os.path.join(r'C:\Users\VinodKumarM\OneDrive - EQUINIX\Projects\fusion_automate_project\fusbq\helix_automation_output', 'RAW', raw_table_name + '.sql'), 'w') as file:
        file.write(colstrraw)
    
    merge_df['mode'] = 'NULLABLE'
    #print(merge_df)
    merge_df['name']=merge_df['COLUMN']
    merge_df['type']=merge_df['bq_type']
    bqschema = merge_df.loc[:, ['name', 'type', 'mode']].to_dict('records')
    
    #print(colstrraw)
    #print(colstrraw)
    with open(os.path.join(r'C:\Users\VinodKumarM\OneDrive - EQUINIX\Projects\fusion_automate_project\fusbq\helix_automation_output', 'JSON', stg_table_name + '.json'), 'w') as file:
        file.write(json.dumps(bqschema, indent=2))


    

                   TABLE                 COLUMN
0  HZ_CUST_SITE_USES_ALL  OBJECT_VERSION_NUMBER
1  HZ_CUST_SITE_USES_ALL               RUN_DATE
2  HZ_CUST_SITE_USES_ALL                    KEY
3  HZ_CUST_SITE_USES_ALL       LAST_UPDATE_DATE
4  HZ_CUST_SITE_USES_ALL        LAST_UPDATED_BY
5  HZ_CUST_SITE_USES_ALL      LAST_UPDATE_LOGIN
6  HZ_CUST_SITE_USES_ALL             CREATED_BY
7  HZ_CUST_SITE_USES_ALL          CREATION_DATE
8  HZ_CUST_SITE_USES_ALL            SITE_USE_ID
     OWNER            COLUMN_NAME             TABLE_NAME DATA_TYPE  \
0   FUSION             ATTRIBUTE2  HZ_CUST_SITE_USES_ALL  VARCHAR2   
1   FUSION            ATTRIBUTE10  HZ_CUST_SITE_USES_ALL  VARCHAR2   
2   FUSION            ATTRIBUTE12  HZ_CUST_SITE_USES_ALL  VARCHAR2   
3   FUSION            ATTRIBUTE13  HZ_CUST_SITE_USES_ALL  VARCHAR2   
4   FUSION            ATTRIBUTE20  HZ_CUST_SITE_USES_ALL  VARCHAR2   
..     ...                    ...                    ...       ...   
91  FUSION          SITE_USE_C